# Setup

## The usual

As always:

1. Clone the lab repository to your computer
1. Open the lab repository in VS Code
1. Open the Julia REPL and activate, then instantiate, the lab environment
1. Make sure you can render: `quarto render template.qmd` in the terminal.
    - If you run into issues, try running `] build IJulia` in the Julia REPL (`]` enters the package manager).
    - If you still have issues, try opening up `blankfile.py`. That should trigger VS Code to give you the option to install the Python extension, which you should do. Then you should be able to open a menu in the bottom right of your screen to select which Python installation you want VS Code to use.


## Load packages


In [ ]:
using CSV
using DataFrames
using DataFramesMeta
using Distributions
using Plots
using StatsPlots
using Unitful

Plots.default(; margin=5Plots.mm)

## Local package


In [ ]:
using Revise
using HouseElevation

In [ ]:
house = let
    haz_fl_dept = CSV.read("data/haz_fl_dept.csv", DataFrame) # read in the file
    desc = "Apartment, living area on one floor, Structure"
    row = @rsubset(haz_fl_dept, :Description == desc)[1, :] # select the row I want
    area = 1033u"ft^2"
    height_above_gauge = 4u"ft"
    House(
        row;
        area=area,
        height_above_gauge=height_above_gauge,
        value_usd=316_298,
    )
end

## Location
This location is near Peir 12 in Galveston of 2016 Strand #8, Galveston, TX 77550, which is a small groundlevel apartment. Values for the square footage and value were sourced from Redfin. The depth damage curve is thus an apartment on one floor in Galveston.


In [ ]:
let
    depths = uconvert.(u"ft", (-7.0u"ft"):(1.0u"inch"):(30.0u"ft"))
    damages = house.ddf.(depths) ./ 100
    damages_1000_usd = damages .* house.value_usd ./ 1000
    scatter(
        depths,
        damages_1000_usd;
        xlabel="Flood Depth",
        ylabel="Damage (Thousand USD)",
        label="$(house.description)\n($(house.source))",
        legend=:bottomright,
        size=(800, 400),
        yformatter=:plain, # prevents scientific notation
    )
end

In [ ]:
let
    elevations = 0u"ft":0.25u"ft":14u"ft"
    costs = [elevation_cost(house, eᵢ) for eᵢ in elevations]
    scatter(
        elevations,
        costs ./ 1_000;
        xlabel="Elevation",
        ylabel="Cost (Thousand USD)",
        label="$(house.description)\n($(house.source))",
        legend=:bottomright,
        size=(800, 400),
        yformatter=:plain, # prevents scientific notation
    )
end

In [ ]:
slr_scenarios = let
    df = CSV.read("data/slr_oddo.csv", DataFrame)
    [Oddo17SLR(a, b, c, tstar, cstar) for (a, b, c, tstar, cstar) in eachrow(df)]
end
println("There are $(length(slr_scenarios)) parameter sets")

In [ ]:
let
    years = 1900:2150
    p = plot(;
        xlabel="Year",
        ylabel="Mean sea-level (ft)\nwith respect to the year 2000",
        label="Oddo et al. (2017)",
        legend=false
    )
    for s in rand(slr_scenarios, 250)
        plot!(p, years, s.(years); color=:lightgrey, alpha=0.5, linewidth=0.5)
    end
    p
end

# Functions that samples storm surge projections and discount rates.


In [ ]:
function draw_surge_distribution()
    μ = rand(Normal(5, 1))
    σ = rand(Exponential(1.5))
    ξ = rand(Normal(0.1, 0.05))
    GeneralizedExtremeValue(μ, σ, ξ)
end

In [ ]:
function draw_discount_rate()
    return rand(Normal(0.1, 0.02))
end

## Modeling Reasoning
The discount rate is modeled after the S&P 500 which has an expected discount rate of 10%. This may flucuate from time to time by approximately 4%, so the standard deviation will be 2%, allowing for the majority of the normal distribution to be with +/- 4% of 10%. 

The surge/flood distribution is hypothetical and based on the deterministic distribution that has been used in previous labs.


In [ ]:
p = ModelParams(
    house=house,
    years=2024:2083
)

In [ ]:
sow = SOW(
    rand(slr_scenarios),
    draw_surge_distribution(),
    draw_discount_rate()
)

In [ ]:
a = Action(5.0u"ft")

In [ ]:
res = run_sim(a, sow, p)

In [ ]:
tempsows = [SOW(rand(slr_scenarios), draw_surge_distribution(), draw_discount_rate()) for _ in 1:10] # for 10 SOWs
sows = []
for i in 1:11
append!(sows, tempsows)
end

n = 0
actions = []
while n < 11
tempactions = [Action(n*1u"ft") for _ in 1:10] # these are all the same
append!(actions, tempactions)
n = n+1
end

results = [run_sim(a, s, p) for (a, s) in zip(actions, sows)]

In [ ]:
df = DataFrame(
    npv=results,
    Δh_ft=[a.Δh_ft for a in actions],
    slr_a=[s.slr.a for s in sows],
    slr_b=[s.slr.b for s in sows],
    slr_c=[s.slr.c for s in sows],
    slr_tstar=[s.slr.tstar for s in sows],
    slr_cstar=[s.slr.cstar for s in sows],
    surge_μ=[s.surge_dist.μ for s in sows],
    surge_σ=[s.surge_dist.σ for s in sows],
    surge_ξ=[s.surge_dist.ξ for s in sows],
    discount_rate=[s.discount_rate for s in sows],
)

In [ ]:
let
    npv = results
    elevated = [a.Δh_ft for a in actions]
    scatter(
        elevated,
        npv;
        xlabel="Elevation Action",
        ylabel="NPV",
        label="The NPV for elevating an apartment at different elevations under 10 different SOW's",
        legend=:bottomright,
        size=(800, 400),
        yformatter=:plain, # prevents scientific notation
    )
end

In [ ]:
let
    npv = results
    discountrate = [s.discount_rate for s in sows]
    scatter(
        discountrate,
        npv;
        xlabel="Discount Rate",
        ylabel="NPV",
        label="The NPV under 10 different SOW's compared to the discount rate (%)",
        legend=:bottomright,
        size=(800, 400),
        yformatter=:plain, # prevents scientific notation
    )
end

In [ ]:
    npv = results
    surge = DataFrame(surge_μ=[s.surge_dist.μ for s in sows],
    surge_σ=[s.surge_dist.σ for s in sows],
    surge_ξ=[s.surge_dist.ξ for s in sows],)
    damage = []
    for i in 1:110
        total = 0
        for n in 1:10000
            sample = rand(GeneralizedExtremeValue(surge[i, 1], surge[i, 2], surge[i, 3]))
            total = total + sample
            global dmean = total/10000
        end
        damage = [damage;dmean]
    end
    scatter(
        damage,
        npv;
        xlabel="Expected flood height from the guage (ft)",
        ylabel="NPV",
        label="The NPV under 10 different SOW's compared to the expected flood height from the gauge",
        legend=:bottomright,
        size=(800, 400),
        yformatter=:plain, # prevents scientific notation
    )

# Discussion/Analysis

Three scatter plots were created to test to what extent the different parameters influenced the final outcome of the NPV. For the analysis, 10 different SOW's were created. Under each of these SOW's 11 actions were performed in each of them by elevating the house from 0 ft to 10 ft. 

The most important parameter was the elevation action. On the graph, there is a very clear cone of uncertainty with the NPV rising the more you elevate, which makes sense as less money is spent on rebuilding each year.

The second most important variable was the expected flood height, which approximates the flood distribution in general. For lower heights, the bulk of the points are near the higher NPV values. As the height increase, however, while you do still find points at high NPV values, they are more spread downwards towards lower NPV values. This makes sense as a lower height will result in lower damages in the future.

For the discount rate, there was no meaningful insight that could be gained from the graph as the points were spread out fairly uniformily. I expected higher discount rates to have higher NPV values as costs to rebuild in the future would result in a lower subtraction from the NPV.

I couldn't think of a satisfying way to analyze sea rise as unlike the other parameters, it is something that changes throughout the SOW. While the different coefficients in the equation could be graphed, I was unsure what to expect due to my unfamiliarity with the equation.

Overall, the analysis shows that it is almost always a good decision to elevate yours house under a range of SOWs. Given unlimited computing power it would make sense to compute many more SOW's and actions as they could better reveal the relationships between the parameters and the NPV. For example, narrowing down on the cone of uncertainty in the NPV vs. Elevation graph. Due to real world computing limitations, though, time to run the program would be the main limiting factor in actually running more SOW's.
